In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.particle import Particle, Neutron
from ATARI import TAZ 

In [2]:
%matplotlib widget

In [3]:
Ta181 = Particle(Z=73, A=181, I=3.5, mass=180.94803, name='Ta181')
Ta_pair = Particle_Pair(isotope = "Ta181",
                        resonance_ladder = pd.DataFrame(),
                        formalism = "XCT",
                        energy_range = [200,350],
                        ac = 0.8127,
                        target=Ta181,
                        projectile=Neutron,
                        l_max = 1
)

# print quant number map up to l_max
Ta_pair.map_quantum_numbers(print_out=True)

# J = 3, <D> = 9.0030 eV, <gn2> = 452.56615 meV, <gg2> = 32.0 meV
# J = 4, <D> = 8.3031 eV, <gn2> = 332.24347 meV, <gg2> = 32.0 meV

# add spin group information for both s-wave resonances
Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615, #46.5,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

Ta_pair.add_spin_group(Jpi='4.0',
                       J_ID=2,
                       D=8.3031,
                       gn2_avg=332.24347, #35.5,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

print()
print("Interrogate spin group attribute:")
print(Ta_pair.spin_groups[3.0])


print()
print("Resonance ladder sampled from resonance statistics:")
print(Ta_pair.sample_resonance_ladder())

Spin group data for all parity
(Jpi, #Chs, l-waves)
(-5.0, 1, [1])
(-4.0, 2, [1, 1])
(-3.0, 2, [1, 1])
(-2.0, 1, [1])
(3.0, 1, [0])
(4.0, 1, [0])

Interrogate spin group attribute:
{'Jpi': 3.0, 'J_ID': 1, '<D>': 9.003, '<gn2>': 452.56615, 'n_dof': 1, '<gg2>': 32.0, 'g_dof': 1000, 'quantiles': {'D01': 1.0184333135415622, 'D99': 21.80043460630884, 'gn01': 0.0710926470659408, 'gn99': 3002.729610372257, 'gg01': 28.765198301747624, 'gg99': 35.42300781927096, 'gt01': 28.836290948813566, 'gt99': 3038.152618191528}, 'chs': 1, 'Ls': [0]}

Resonance ladder sampled from resonance statistics:
             E         Gg         Gn1  J_ID        gg2          gn2  Jpi    L
0   200.765336  65.250647   26.471740   1.0  32.625323   526.138416  3.0  0.0
1   203.407471  67.546704   47.971293   1.0  33.773352   947.239630  3.0  0.0
2   208.069254  64.058825    0.103132   1.0  32.029412     2.013493  3.0  0.0
3   217.888881  63.431051    0.261410   1.0  31.715525     4.987305  3.0  0.0
4   225.004553  66.841

In [4]:
# Ta_pair.resonance_ladder

In [5]:
PT_probs, PT_log_likelihood = TAZ.PTBayes(Ta_pair, lvl_dens_false=0.0)
print(PT_probs)

[[0.49376312 0.50623688 0.        ]
 [0.53582297 0.46417703 0.        ]
 [0.44160365 0.55839635 0.        ]
 [0.44189708 0.55810292 0.        ]
 [0.44279464 0.55720536 0.        ]
 [0.44142111 0.55857889 0.        ]
 [0.44208305 0.55791695 0.        ]
 [0.56308965 0.43691035 0.        ]
 [0.45275344 0.54724656 0.        ]
 [0.44843262 0.55156738 0.        ]
 [0.46926853 0.53073147 0.        ]
 [0.47892571 0.52107429 0.        ]
 [0.44890577 0.55109423 0.        ]
 [0.45325174 0.54674826 0.        ]
 [0.5823259  0.4176741  0.        ]
 [0.50383399 0.49616601 0.        ]
 [0.45806985 0.54193015 0.        ]
 [0.46448732 0.53551268 0.        ]
 [0.78003663 0.21996337 0.        ]
 [0.54748205 0.45251795 0.        ]
 [0.46187281 0.53812719 0.        ]
 [0.49707567 0.50292433 0.        ]
 [0.44459547 0.55540453 0.        ]
 [0.44712821 0.55287179 0.        ]
 [0.4766738  0.5233262  0.        ]
 [0.4423958  0.5576042  0.        ]
 [0.4454658  0.5545342  0.        ]
 [0.45566953 0.54433047 0.  

In [6]:
lvl_dens = []
for Jpi, spingroup in Ta_pair.spin_groups.items():
    lvl_dens.append(1/spingroup['<D>'])
lvl_dens = np.array(lvl_dens)
distributions = TAZ.Distributions.wigner(lvl_dens)
runMaster = TAZ.RunMaster(Ta_pair.resonance_ladder['E'].to_numpy(), Ta_pair.energy_range, distributions, false_dens=0.0, Prior=PT_probs, log_likelihood_prior=PT_log_likelihood)
Posterior = runMaster.WigBayes(verbose=True)

print()
print(Posterior)

Preparing level-spacings
Finished level-spacing calculations
Finished CP calculation
Finished WigBayes calculation

[[ 4.67364317e-01  5.32635688e-01 -5.05462139e-09]
 [ 5.22236896e-01  4.77763109e-01 -5.05462117e-09]
 [ 4.90995906e-01  5.09004099e-01 -5.05462139e-09]
 [ 4.91482916e-01  5.08517089e-01 -5.05462117e-09]
 [ 4.68383897e-01  5.31616108e-01 -5.05462117e-09]
 [ 5.15814772e-01  4.84185233e-01 -5.05462117e-09]
 [ 3.69372287e-01  6.30627718e-01 -5.05462117e-09]
 [ 6.01038773e-01  3.98961232e-01 -5.05462117e-09]
 [ 3.96914144e-01  6.03085861e-01 -5.05462117e-09]
 [ 4.74506525e-01  5.25493480e-01 -5.05462139e-09]
 [ 5.13735241e-01  4.86264764e-01 -4.71629535e-09]
 [ 4.35369046e-01  5.64630959e-01 -4.70800243e-09]
 [ 4.80146561e-01  5.19853444e-01 -4.70412531e-09]
 [ 4.62086136e-01  5.37913869e-01 -4.61601291e-09]
 [ 5.63857344e-01  4.36142660e-01 -4.35345271e-09]
 [ 5.02726001e-01  4.97274003e-01 -4.34923852e-09]
 [ 4.83464388e-01  5.16535617e-01 -4.31485558e-09]
 [ 3.80430243e-01